Import libraries and upload data from nflverse github

In [1]:
#Import package
from urllib.request import urlretrieve

#Import Pandas
import pandas as pd

#Import Numpy
import numpy as np

#Import Matplotlib
import matplotlib.pyplot as plt

#Import Seaborn
import seaborn as sns

import sys
!{sys.executable} -m pip install python-dotenv requests

#Import requests
import requests

#Import datetime
from datetime import datetime

#Import time
import time

#Import beautifulsoup
from bs4 import BeautifulSoup

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


/Users/jonathan/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
#Assign Url of the file: Url
Url = "https://raw.githubusercontent.com/nflverse/nfldata/refs/heads/master/data/games.csv"

#Save the file locally
urlretrieve(Url, 'games.csv')

#Read file into a DataFrame and print head
games = pd.read_csv('games.csv')
print(games.head())

           game_id  season game_type  week     gameday weekday gametime  \
0  1999_01_MIN_ATL    1999       REG     1  1999-09-12  Sunday      NaN   
1   1999_01_KC_CHI    1999       REG     1  1999-09-12  Sunday      NaN   
2  1999_01_PIT_CLE    1999       REG     1  1999-09-12  Sunday      NaN   
3   1999_01_OAK_GB    1999       REG     1  1999-09-12  Sunday      NaN   
4  1999_01_BUF_IND    1999       REG     1  1999-09-12  Sunday      NaN   

  away_team  away_score home_team  ...  wind  away_qb_id  home_qb_id  \
0       MIN        17.0       ATL  ...   NaN  00-0003761  00-0002876   
1        KC        17.0       CHI  ...  12.0  00-0006300  00-0010560   
2       PIT        43.0       CLE  ...  12.0  00-0015700  00-0004230   
3       OAK        24.0        GB  ...  10.0  00-0005741  00-0005106   
4       BUF        14.0       IND  ...   NaN  00-0005363  00-0010346   

         away_qb_name    home_qb_name          away_coach    home_coach  \
0  Randall Cunningham  Chris Chandler    

In [3]:
#Print info of the data
games.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6991 entries, 0 to 6990
Data columns (total 46 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   game_id           6991 non-null   object 
 1   season            6991 non-null   int64  
 2   game_type         6991 non-null   object 
 3   week              6991 non-null   int64  
 4   gameday           6991 non-null   object 
 5   weekday           6991 non-null   object 
 6   gametime          6732 non-null   object 
 7   away_team         6991 non-null   object 
 8   away_score        6990 non-null   float64
 9   home_team         6991 non-null   object 
 10  home_score        6990 non-null   float64
 11  location          6991 non-null   object 
 12  result            6990 non-null   float64
 13  total             6990 non-null   float64
 14  overtime          6990 non-null   float64
 15  old_game_id       6991 non-null   int64  
 16  gsis              6990 non-null   float64


In [4]:
#Shape of the data
games.shape

(6991, 46)

In [5]:
#print value counts of location column to check if games are duplicated home/away
games.location.value_counts()

location
Home       6905
Neutral      86
Name: count, dtype: int64

More neutral games here than just superbowls, what are the other neutral games?

In [6]:
#Determine what Neutral games are
#Filter for neutral games and display stadium info, and game type
neutral_games = games[games.location == 'Neutral'][['game_id', 'stadium', 'game_type']]
print(neutral_games)


              game_id                  stadium game_type
258   1999_21_STL_TEN             Georgia Dome        SB
517   2000_21_BAL_NYG    Raymond James Stadium        SB
776    2001_21_STL_NE      Louisiana Superdome        SB
1043   2002_21_OAK_TB         Qualcomm Stadium        SB
1159   2003_08_MIA_SD        Sun Devil Stadium       REG
...               ...                      ...       ...
6785  2024_06_JAX_CHI        Tottenham Stadium       REG
6799   2024_07_NE_JAX          Wembley Stadium       REG
6845  2024_10_NYG_CAR            Allianz Arena       REG
6983   2024_19_MIN_LA       State Farm Stadium        WC
6990   2024_22_KC_PHI  Mercedes-Benz Superdome        SB

[86 rows x 3 columns]


It seems like most of these are games played abroad, and 2 games played to avoid wildfires, 2003 MIA @ SD regular season game and 2024 MIN @ LA wildcard game, however SD no longer has a team, I should check what teams are in this data

In [7]:
#How many teams are there?
games.home_team.value_counts()

home_team
NE     239
PHI    232
KC     230
GB     226
PIT    225
SEA    224
IND    223
DEN    221
NO     221
SF     221
TB     221
BAL    220
BUF    219
CHI    218
ATL    217
TEN    217
DAL    217
MIN    217
NYG    216
CAR    216
CIN    216
ARI    215
WAS    214
JAX    214
DET    213
MIA    213
NYJ    211
CLE    210
HOU    194
OAK    173
SD     149
STL    141
LA      80
LAC     66
LV      42
Name: count, dtype: int64

Teams have moved cities since 1999, when the data begins, OAK to LV, STL to LA, and SD to LAC, these teams should be combinded

In [8]:
#Combine teams that have moved cities for home_team and away_team
games.home_team.replace({'OAK': 'LV', 'STL': 'LA', 'SD': 'LAC'}, inplace=True)
games.away_team.replace({'OAK': 'LV', 'STL': 'LA', 'SD': 'LAC'}, inplace=True)

#check value counts of home_team and away_team
print(games.home_team.value_counts())
print(games.away_team.value_counts())

home_team
NE     239
PHI    232
KC     230
GB     226
PIT    225
SEA    224
IND    223
DEN    221
NO     221
SF     221
LA     221
TB     221
BAL    220
BUF    219
CHI    218
MIN    217
DAL    217
ATL    217
TEN    217
CIN    216
CAR    216
NYG    216
LV     215
LAC    215
ARI    215
WAS    214
JAX    214
DET    213
MIA    213
NYJ    211
CLE    210
HOU    194
Name: count, dtype: int64
away_team
BAL    232
GB     226
IND    225
SEA    225
PIT    224
NE     223
PHI    223
NYG    222
SF     222
LA     222
NYJ    221
TEN    221
MIN    220
CAR    219
KC     219
LAC    219
DAL    218
NO     218
TB     218
ATL    217
WAS    217
CIN    217
JAX    216
MIA    216
DEN    215
DET    215
ARI    215
BUF    215
CLE    214
LV     214
CHI    211
HOU    192
Name: count, dtype: int64


/var/folders/pm/v1wywnps12sf4vmw4zxrydwr0000gn/T/ipykernel_31809/2999817110.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  games.home_team.replace({'OAK': 'LV', 'STL': 'LA', 'SD': 'LAC'}, inplace=True)
/var/folders/pm/v1wywnps12sf4vmw4zxrydwr0000gn/T/ipykernel_31809/2999817110.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on wh

In [9]:
#Check for null values in each column
print(games.isnull().sum().sort_values(ascending=False))

nfl_detail_id       6719
ftn                 5615
pff                 2699
under_odds          1984
over_odds           1984
home_moneyline      1981
away_moneyline      1981
away_spread_odds    1981
home_spread_odds    1981
temp                1975
wind                1975
gametime             259
surface               43
referee                2
total                  1
away_score             1
home_score             1
result                 1
overtime               1
gsis                   1
div_game               0
roof                   0
game_id                0
home_qb_id             0
away_qb_name           0
home_qb_name           0
away_coach             0
home_coach             0
stadium_id             0
away_qb_id             0
home_rest              0
total_line             0
spread_line            0
season                 0
away_rest              0
espn                   0
pfr                    0
old_game_id            0
location               0
home_team              0


In [10]:
#Display row with missing scores
print(games[games.home_score.isnull() | games.away_score.isnull()][['game_id', 'home_team', 'away_team', 'home_score', 'away_score', 'gameday']])


             game_id home_team away_team  home_score  away_score     gameday
6990  2024_22_KC_PHI       PHI        KC         NaN         NaN  2025-02-09


The 2024 SB has not been played yet, so it is not a problem that the scores are missing.

Missing values for gambling lines can be ignored for now, as the goal for the model is to predict the outcome of the game.
They may be useful later, so I will keep them to the side.

PFF and PTN are ID numbers for premium databases, so they will be ignored in this project.

Game time is missing just for the 1999 season, so it is not a problem that it is missing.

In [11]:
#Check for missing wind and temperature values
print("Rows with missing wind values:")
print(games[games.wind.isnull()][['game_id', 'home_team', 'away_team', 'gameday', 'wind']])
print("\nRows with missing temperature values:")
print(games[games.temp.isnull()][['game_id', 'home_team', 'away_team', 'gameday', 'temp']])


Rows with missing wind values:
              game_id home_team away_team     gameday  wind
0     1999_01_MIN_ATL       ATL       MIN  1999-09-12   NaN
4     1999_01_BUF_IND       IND       BUF  1999-09-12   NaN
6      1999_01_CAR_NO        NO       CAR  1999-09-12   NaN
9     1999_01_DET_SEA       SEA       DET  1999-09-12   NaN
10    1999_01_BAL_STL        LA       BAL  1999-09-12   NaN
...               ...       ...       ...         ...   ...
6977  2024_18_MIN_DET       DET       MIN  2025-01-05   NaN
6978  2024_19_LAC_HOU       HOU       LAC  2025-01-11   NaN
6983   2024_19_MIN_LA        LA       MIN  2025-01-13   NaN
6985  2024_20_WAS_DET       DET       WAS  2025-01-18   NaN
6990   2024_22_KC_PHI       PHI        KC  2025-02-09   NaN

[1975 rows x 5 columns]

Rows with missing temperature values:
              game_id home_team away_team     gameday  temp
0     1999_01_MIN_ATL       ATL       MIN  1999-09-12   NaN
4     1999_01_BUF_IND       IND       BUF  1999-09-12   NaN
6    

Some of these stadiums are domes, so the wind and temperature may not be relevant data will be missing for these. If they are a dome I will impute 0 for wind and 70 for temp, the average indoor nfl stadium temperature.

In [12]:
#Check unique values for roof
print("\nUnique values for roof:")
print(games.roof.value_counts())



Unique values for roof:
roof
outdoors    5140
dome        1144
closed       579
open         128
Name: count, dtype: int64


In [13]:
#Display rows where roof is open or closed
print("\nRows where roof is open or closed:")
print(games[games.roof.isin(['open', 'closed'])][['game_id', 'home_team', 'away_team', 'gameday', 'roof']])



Rows where roof is open or closed:
              game_id home_team away_team     gameday    roof
791   2002_01_DAL_HOU       HOU       DAL  2002-09-08  closed
810   2002_03_IND_HOU       HOU       IND  2002-09-22    open
853   2002_06_BUF_HOU       HOU       BUF  2002-10-13  closed
898   2002_09_CIN_HOU       HOU       CIN  2002-11-03  closed
923   2002_11_JAX_HOU       HOU       JAX  2002-11-17    open
...               ...       ...       ...         ...     ...
6965  2024_18_WAS_DAL       DAL       WAS  2025-01-05  closed
6967  2024_18_JAX_IND       IND       JAX  2025-01-05  closed
6972   2024_18_SF_ARI       ARI        SF  2025-01-05  closed
6978  2024_19_LAC_HOU       HOU       LAC  2025-01-11  closed
6983   2024_19_MIN_LA        LA       MIN  2025-01-13  closed

[707 rows x 5 columns]


In [14]:
# Impute wind and temp values for indoor stadiums
games.loc[(games.wind.isnull()) & (games.roof.isin(['dome', 'closed'])), 'wind'] = 0
games.loc[(games.temp.isnull()) & (games.roof.isin(['dome', 'closed'])), 'temp'] = 70

# Check remaining missing values
print("\nRemaining rows with missing wind values:")
print(games[games.wind.isnull()][['game_id', 'home_team', 'away_team', 'gameday', 'wind', 'roof']])
print("\nRemaining rows with missing temperature values:") 
print(games[games.temp.isnull()][['game_id', 'home_team', 'away_team', 'gameday', 'temp', 'roof']])



Remaining rows with missing wind values:
              game_id home_team away_team     gameday  wind      roof
334   2000_06_BUF_MIA       MIA       BUF  2000-10-08   NaN  outdoors
467    2000_15_CAR_KC        KC       CAR  2000-12-10   NaN  outdoors
810   2002_03_IND_HOU       HOU       IND  2002-09-22   NaN      open
923   2002_11_JAX_HOU       HOU       JAX  2002-11-17   NaN      open
949   2002_12_NYG_HOU       HOU       NYG  2002-11-24   NaN      open
...               ...       ...       ...         ...   ...       ...
6707   2024_01_GB_PHI       PHI        GB  2024-09-06   NaN  outdoors
6845  2024_10_NYG_CAR       CAR       NYG  2024-11-10   NaN  outdoors
6891  2024_13_HOU_JAX       JAX       HOU  2024-12-01   NaN  outdoors
6918  2024_15_NYJ_JAX       JAX       NYJ  2024-12-15   NaN  outdoors
6961   2024_17_DET_SF        SF       DET  2024-12-30   NaN  outdoors

[252 rows x 6 columns]

Remaining rows with missing temperature values:
              game_id home_team away_team    

For remaining missing values I will pull historical weather data from an API and impute the values.
First I need to fill in the missing game times for the 1999 season.

In [15]:
# Check for missing game times
print("Rows with missing game times:")
print(games[games.gametime.isnull()][['game_id', 'home_team', 'away_team', 'gameday', 'gametime']])


Rows with missing game times:
             game_id home_team away_team     gameday gametime
0    1999_01_MIN_ATL       ATL       MIN  1999-09-12      NaN
1     1999_01_KC_CHI       CHI        KC  1999-09-12      NaN
2    1999_01_PIT_CLE       CLE       PIT  1999-09-12      NaN
3     1999_01_OAK_GB        GB        LV  1999-09-12      NaN
4    1999_01_BUF_IND       IND       BUF  1999-09-12      NaN
..               ...       ...       ...         ...      ...
254  1999_19_TEN_IND       IND       TEN  2000-01-16      NaN
255  1999_19_MIN_STL        LA       MIN  2000-01-16      NaN
256  1999_20_TEN_JAX       JAX       TEN  2000-01-23      NaN
257   1999_20_TB_STL        LA        TB  2000-01-23      NaN
258  1999_21_STL_TEN       TEN        LA  2000-01-30      NaN

[259 rows x 5 columns]


To get times I will be bringing in another dataframe from pro football reference and taking times from there.

In [16]:
url2 = 'https://www.pro-football-reference.com/years/1999/games.htm#games'
# Send GET request and create BeautifulSoup object
import requests
response = requests.get(url2)
soup = BeautifulSoup(response.content, 'html.parser')

# Find the main games table
table = soup.find('table', id='games')

# Initialize lists to store data
data = []

# Extract data from table rows
for row in table.find_all('tr')[1:]:  # Skip header row
    cols = row.find_all(['th', 'td'])
    if len(cols) > 0:
        week = cols[0].text.strip()
        day = cols[1].text.strip()
        date = cols[2].text.strip()
        time = cols[3].text.strip()
        winner = cols[4].text.strip()
        loser = cols[6].text.strip()
        pts_w = cols[7].text.strip()
        pts_l = cols[8].text.strip()
        yds_w = cols[9].text.strip()
        to_w = cols[10].text.strip()
        yds_l = cols[11].text.strip()
        to_l = cols[12].text.strip()
        
        data.append([week, day, date, time, winner, loser, pts_w, pts_l, yds_w, to_w, yds_l, to_l])

# Create DataFrame
games1999 = pd.DataFrame(data, columns=['Week', 'Day', 'Date', 'Time', 'Winner', 'Loser', 
                                      'PtsW', 'PtsL', 'YdsW', 'TOW', 'YdsL', 'TOL'])
 #print games1999 head
print(games1999.head())
print(games1999.info())
print(games1999.shape)

  Week  Day        Date    Time              Winner                Loser  \
0    1  Sun  1999-09-12  1:00PM      Dallas Cowboys  Washington Redskins   
1    1  Sun  1999-09-12  1:02PM  Indianapolis Colts        Buffalo Bills   
2    1  Sun  1999-09-12  1:02PM  New Orleans Saints    Carolina Panthers   
3    1  Sun  1999-09-12  1:02PM       Chicago Bears   Kansas City Chiefs   
4    1  Sun  1999-09-12  1:02PM   Green Bay Packers      Oakland Raiders   

       PtsW PtsL YdsW  TOW YdsL  TOL  
0  boxscore   41   35  541    3  504  
1  boxscore   31   14  393    3  321  
2  boxscore   19   10  255    0  246  
3  boxscore   20   17  388    2  349  
4  boxscore   28   24  409    4  363  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276 entries, 0 to 275
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Week    276 non-null    object
 1   Day     276 non-null    object
 2   Date    276 non-null    object
 3   Time    276 non-nul

In [17]:
# Create dictionary for team name to abbreviation mapping
team_mapping = {
    'Arizona Cardinals': 'ARI', 'Atlanta Falcons': 'ATL', 'Baltimore Ravens': 'BAL', 
    'Buffalo Bills': 'BUF', 'Carolina Panthers': 'CAR', 'Chicago Bears': 'CHI',
    'Cincinnati Bengals': 'CIN', 'Cleveland Browns': 'CLE', 'Dallas Cowboys': 'DAL',
    'Denver Broncos': 'DEN', 'Detroit Lions': 'DET', 'Green Bay Packers': 'GB',
    'Houston Texans': 'HOU', 'Indianapolis Colts': 'IND', 'Jacksonville Jaguars': 'JAX',
    'Kansas City Chiefs': 'KC', 'Los Angeles Chargers': 'LAC', 'Los Angeles Rams': 'LA',
    'Las Vegas Raiders': 'LV', 'Miami Dolphins': 'MIA', 'Minnesota Vikings': 'MIN',
    'New England Patriots': 'NE', 'New Orleans Saints': 'NO', 'New York Giants': 'NYG',
    'New York Jets': 'NYJ', 'Philadelphia Eagles': 'PHI', 'Pittsburgh Steelers': 'PIT',
    'San Francisco 49ers': 'SF', 'Seattle Seahawks': 'SEA', 'Tampa Bay Buccaneers': 'TB',
    'Tennessee Titans': 'TEN', 'Washington Commanders': 'WAS',
    # Add historical team names
    'Oakland Raiders': 'LV', 'San Diego Chargers': 'LAC', 'St. Louis Rams': 'LA',
    'Washington Redskins': 'WAS', 'Washington Football Team': 'WAS'
}

# Replace team names with abbreviations
games1999['Winner'] = games1999['Winner'].replace(team_mapping)
games1999['Loser'] = games1999['Loser'].replace(team_mapping)

# Verify the changes
print("\nAfter team name conversion:")
print(games1999[['Winner', 'Loser']].head())



After team name conversion:
  Winner Loser
0    DAL   WAS
1    IND   BUF
2     NO   CAR
3    CHI    KC
4     GB    LV


In [18]:
#Check unique values of Time
print("Unique time values:")
print(games1999['Time'].unique())

# Define a function to convert time to 24-hour format
def convert_to_24hr(time_str):
    try:
        # Remove any extra spaces
        time_str = time_str.strip()
        # Convert to datetime using pandas
        time_obj = pd.to_datetime(time_str).strftime('%H:%M')
        return time_obj
    except:
        return time_str  # Return original if conversion fails

# Apply the conversion
games1999['Time'] = games1999['Time'].apply(convert_to_24hr)

# Check the results
print("\nAfter conversion:")
print(games1999['Time'].head())

Unique time values:
['1:00PM' '1:02PM' '1:03PM' '1:04PM' '4:15PM' '4:16PM' '8:26PM' '9:07PM'
 'Time' '1:01PM' '4:05PM' '8:22PM' '8:23PM' '8:08PM' '1:05PM' '4:18PM'
 '8:24PM' '4:06PM' '4:17PM' '9:09PM' '8:21PM' '1:06PM' '5:15PM' '2:00PM'
 '2:55PM' '8:20PM' '9:10PM' '12:00PM' '12:03PM' '9:08PM' '5:05PM'
 '11:07PM' '12:35PM' '1:10PM' '4:07PM' '1:15PM' '9:05PM' '2:05PM'
 '12:41PM' '2:15PM' '6:23PM' '9:15PM' '3:00PM' '' '12:32PM' '12:42PM'
 '4:22PM' '6:25PM']

After conversion:
0    13:00
1    13:02
2    13:02
3    13:02
4    13:02
Name: Time, dtype: object


In [19]:
# Get unique values from gametime column
print("Unique game times:")
print(games['gametime'].unique())



Unique game times:
[nan '13:00' '16:05' '16:15' '20:20' '09:00' '12:00' '15:05' '15:15'
 '13:05' '12:30' '16:00' '18:18' '20:30' '13:30' '17:00' '16:30' '20:00'
 '18:40' '15:00' '18:30' '20:15' '14:00' '16:45' '18:45' '18:25' '19:30'
 '19:00' '22:15' '16:25' '16:35' '20:25' '19:10' '22:25' '20:40' '18:31'
 '16:40' '18:32' '21:15' '19:20' '23:30' '20:10' '21:20' '09:30' '18:50'
 '22:20' '21:30' '16:36' '18:41' '19:15' '22:10' '19:05' '21:00' '15:40'
 '20:05' '16:50']


This will be easier to work with if I create a winner column based on scores and teams.

In [20]:
# Create Winner column based on scores and teams
games['Winner'] = games.apply(lambda row: 
    row['home_team'] if row['home_score'] > row['away_score']
    else row['away_team'] if row['away_score'] > row['home_score']
    else 'Tie', axis=1)

# Verify the results
print("\nSample of Winners:")
print(games[['home_team', 'away_team', 'home_score', 'away_score', 'Winner']].head())



Sample of Winners:
  home_team away_team  home_score  away_score Winner
0       ATL       MIN        14.0        17.0    MIN
1       CHI        KC        20.0        17.0    CHI
2       CLE       PIT         0.0        43.0    PIT
3        GB        LV        28.0        24.0     GB
4       IND       BUF        31.0        14.0    IND


In [21]:
# Create a mapping of game details to times from games1999
time_mapping = games1999.set_index(['Winner', 'Date'])['Time'].to_dict()

# Function to look up time based on winner and date
def get_time(row):
    if pd.isna(row['gametime']):
        try:
            # Try to find matching game in 1999 data
            return time_mapping.get((row['Winner'], row['gameday']))
        except:
            return row['gametime']
    return row['gametime']

# Fill missing times
games['gametime'] = games.apply(get_time, axis=1)

# Check results
print("\nMissing times after filling:")
print(games['gametime'].isna().sum())
print("\nSample of filled times:")
print(games[['gameday', 'Winner', 'gametime']].head())



Missing times after filling:
0

Sample of filled times:
      gameday Winner gametime
0  1999-09-12    MIN    16:15
1  1999-09-12    CHI    13:02
2  1999-09-12    PIT    20:26
3  1999-09-12     GB    13:02
4  1999-09-12    IND    13:02


Now the times are set. We need to find which stadiums have missing wind and temp data, and use the lat/long of each stadium to get weather data from the API with our dates and times.

In [29]:
print(games.isnull().sum().sort_values(ascending=False))

nfl_detail_id       6719
ftn                 5615
pff                 2699
over_odds           1984
under_odds          1984
home_spread_odds    1981
away_spread_odds    1981
home_moneyline      1981
away_moneyline      1981
wind                 252
temp                 252
surface               43
referee                2
result                 1
total                  1
overtime               1
gsis                   1
away_score             1
home_score             1
roof                   0
div_game               0
game_id                0
home_qb_id             0
away_qb_name           0
home_qb_name           0
away_coach             0
home_coach             0
stadium_id             0
stadium                0
away_qb_id             0
home_rest              0
total_line             0
spread_line            0
season                 0
away_rest              0
espn                   0
pfr                    0
old_game_id            0
location               0
home_team              0
